In [13]:
import ast
import json
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import os
import pandas as pd


Put Files here

In [14]:
cluster_folder = "Mapping_data/companies"
matches_folder = "Mapping_data/companies_mapped"
output_file = "Matching.xlsx"

This is for Clustered Matches

In [15]:
# loop through all the files in the folder
for file in os.listdir(cluster_folder):

    with open(cluster_folder+"/"+file, 'r') as f:
        data = json.load(f)

    # Initialize an empty list for storing final rows
    final_rows = []

    # Parse the JSON data
    for stringified_keys, list_of_data_points in data.items():
        try:
            parsed_keys = ast.literal_eval(stringified_keys)
        except Exception as e:
            print(f"Failed to parse stringified keys: {e}")
            continue

        parsed_keys = {f"{key}_original": value for key, value in parsed_keys.items()}

        for data_point in list_of_data_points:
            combined_data = {**parsed_keys, **data_point}
            # combined_data['Company_ticker'] = ticker
            final_rows.append(combined_data)

    # Create a DataFrame
    df = pd.DataFrame(final_rows)

    # Load existing workbook and create a new worksheet
    wb = load_workbook(output_file)
    ws = wb.create_sheet(title=file[:-5]+" Clustered Matches")

    # Write headers
    for col_num, column_title in enumerate(df.columns, 1):
        col_letter = get_column_letter(col_num)
        ws[f"{col_letter}1"] = column_title

    # Write data to worksheet
    for row_index, row in df.iterrows():
        for col_num, (col_name, cell_value) in enumerate(row.items(), 1):
            col_letter = get_column_letter(col_num)
            ws[f"{col_letter}{row_index + 2}"] = cell_value

    # Save workbook
    wb.save(output_file)


This is for mapped Matches

In [16]:
# loop through all the files in the folder
for file in os.listdir(matches_folder):
    # Load JSON data
    with open(matches_folder+"/"+file, 'r') as f:
        data = json.load(f)

    # Initialize an empty list for storing final rows
    final_rows = []

    # Parse the JSON data

    for stringified_keys, correct_data_point in data.items():
        try:
            parsed_keys = ast.literal_eval(stringified_keys)
        except Exception as e:
            print(f"Failed to parse stringified keys: {e}")
            continue

        parsed_keys = {f"{key}_original": value for key, value in parsed_keys.items()}
        combined_data = {**parsed_keys, **correct_data_point}
        final_rows.append(combined_data)

    # Create a DataFrame
    df = pd.DataFrame(final_rows)

    # Create a workbook and add a worksheet
    wb = load_workbook(output_file)
    ws = wb.create_sheet(title=file[:-5])

    # Write headers
    for col_num, column_title in enumerate(df.columns, 1):
        col_letter = get_column_letter(col_num)
        ws[f"{col_letter}1"] = column_title

    # Write data to worksheet
    for row_index, row in df.iterrows():
        for col_num, (col_name, cell_value) in enumerate(row.items(), 1):
            col_letter = get_column_letter(col_num)
            ws[f"{col_letter}{row_index + 2}"] = cell_value

    # Save workbook
    wb.save(output_file)